<a href="https://colab.research.google.com/github/Indresh0007/PySpark-Indresh/blob/main/Advance_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RDD

In [ ]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("MySparkApp").setMaster("local[*]")
sc = SparkContext(conf=conf)

rdd = sc.parallelize([1,2,3,4,5])
rdd_mapped = rdd.map(lambda x: x*2)
print(rdd_mapped.collect())

rdd_filtered = rdd.filter(lambda x: x%2 == 0)
print(rdd_filtered.collect())

sc.stop()


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=count app, master=local) created by __init__ at /tmp/ipython-input-1-3322327215.py:3 

In [31]:
from pyspark import SparkContext, SparkConf

# 🔹 Stop any existing SparkContext
try:
    sc.stop()
except:
    pass

conf = SparkConf().setAppName("MySparkApp").setMaster("local[*]")
sc = SparkContext(conf=conf)

rdd = sc.parallelize([1,2,3,4,5])

#transformation Map
rdd_mapped = rdd.map(lambda x: x*2)
print(rdd_mapped.collect())

#transformation Filter
rdd_filtered = rdd.filter(lambda x: x%2 == 0)
print(rdd_filtered.collect())

#transformation Reduce
rd_reduce = rdd.reduce(lambda x,y: x+y)
print(rd_reduce)

#transformation: Flatmap
rdd =sc.parallelize([1,2,3])
#create a range of numbers for each element
rdd_flatmap = rdd.flatMap(lambda x: range(x,x+4))
#collect the result
print(rdd_flatmap.collect())

rdd=sc.parallelize(["hello","world"])
rdd_flatmap = rdd.flatMap(lambda x: list(x))
print(rdd_flatmap.collect())

rdd=sc.parallelize([[1,2],[3,4],[5]])
rdd_flatmap = rdd.flatMap(lambda x: x)
print(rdd_flatmap.collect())

rdd = sc.parallelize([1,2,3,4,5])
rdd_group =rdd.groupBy(lambda x: "even" if x%2 == 0 else "odd")
print([(key, list(value)) for key, value in rdd_group.collect()])


sc.stop()  #  Stop after work


[2, 4, 6, 8, 10]
[2, 4]
15
[1, 2, 3, 4, 2, 3, 4, 5, 3, 4, 5, 6]
['h', 'e', 'l', 'l', 'o', 'w', 'o', 'r', 'l', 'd']
[1, 2, 3, 4, 5]
[('even', [2, 4]), ('odd', [1, 3, 5])]


## DataFrame

In [32]:
data = [
    (1, "John", "HR", 5000),
    (2, "Jane", "IT", 8000),
    (3, "Mike", "IT", 6000),
    (4, "Sara", "Finance", 7000),
    (5, "David", "HR", 5500)
]

from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder.appName("DataFrameOperations").getOrCreate()

# Define column names
columns = ["ID", "Name", "Department", "Salary"]

# Create a DataFrame from the sample data
df = spark.createDataFrame(data, columns)

# Show the DataFrame
df.show()

+---+-----+----------+------+
| ID| Name|Department|Salary|
+---+-----+----------+------+
|  1| John|        HR|  5000|
|  2| Jane|        IT|  8000|
|  3| Mike|        IT|  6000|
|  4| Sara|   Finance|  7000|
|  5|David|        HR|  5500|
+---+-----+----------+------+



In [33]:
df.select("Name","Salary").show()

+-----+------+
| Name|Salary|
+-----+------+
| John|  5000|
| Jane|  8000|
| Mike|  6000|
| Sara|  7000|
|David|  5500|
+-----+------+



In [34]:
df.filter(df["Salary"] >6000).show()

+---+----+----------+------+
| ID|Name|Department|Salary|
+---+----+----------+------+
|  2|Jane|        IT|  8000|
|  4|Sara|   Finance|  7000|
+---+----+----------+------+



In [41]:
from pyspark.sql.functions import col
df=df.withColumn("Bonus",df["Salary"]*0.1)
df.show()

+---+-----+----------+------+-----+
| ID| Name|Department|Salary|Bonus|
+---+-----+----------+------+-----+
|  1| John|        HR|  5000|500.0|
|  2| Jane|        IT|  8000|800.0|
|  3| Mike|        IT|  6000|600.0|
|  4| Sara|   Finance|  7000|700.0|
|  5|David|        HR|  5500|550.0|
+---+-----+----------+------+-----+



In [49]:
df=df.drop("Bonus")
df.show()

+---+-----+----------+------+
| ID| Name|Department|Salary|
+---+-----+----------+------+
|  1| John|        HR|  5000|
|  2| Jane|        IT|  8000|
|  3| Mike|        IT|  6000|
|  4| Sara|   Finance|  7000|
|  5|David|        HR|  5500|
+---+-----+----------+------+



In [37]:
df=df.withColumnRenamed("Salary","Salary_After_tax")
df.show()


+---+-----+----------+----------------+
| ID| Name|Department|Salary_After_tax|
+---+-----+----------+----------------+
|  1| John|        HR|            5000|
|  2| Jane|        IT|            8000|
|  3| Mike|        IT|            6000|
|  4| Sara|   Finance|            7000|
|  5|David|        HR|            5500|
+---+-----+----------+----------------+



In [40]:
df=df.withColumnRenamed("Salary_After_tax","Salary")
df.show()


+---+-----+----------+------+
| ID| Name|Department|Salary|
+---+-----+----------+------+
|  1| John|        HR|  5000|
|  2| Jane|        IT|  8000|
|  3| Mike|        IT|  6000|
|  4| Sara|   Finance|  7000|
|  5|David|        HR|  5500|
+---+-----+----------+------+



In [42]:
df.groupBy("Department").agg(avg("Salary").alias("Avg-Salary")).show()

+----------+----------+
|Department|Avg-Salary|
+----------+----------+
|        HR|    5250.0|
|        IT|    7000.0|
|   Finance|    7000.0|
+----------+----------+



In [45]:
df.groupBy("Department").count().show()

+----------+-----+
|Department|count|
+----------+-----+
|        HR|    2|
|        IT|    2|
|   Finance|    1|
+----------+-----+



In [50]:
df.sort(df["Salary"].desc()).show()

+---+-----+----------+------+
| ID| Name|Department|Salary|
+---+-----+----------+------+
|  2| Jane|        IT|  8000|
|  4| Sara|   Finance|  7000|
|  3| Mike|        IT|  6000|
|  5|David|        HR|  5500|
|  1| John|        HR|  5000|
+---+-----+----------+------+



In [53]:
data_list=df.collect()

for row in data_list:
  print(row)

Row(ID=1, Name='John', Department='HR', Salary=5000)
Row(ID=2, Name='Jane', Department='IT', Salary=8000)
Row(ID=3, Name='Mike', Department='IT', Salary=6000)
Row(ID=4, Name='Sara', Department='Finance', Salary=7000)
Row(ID=5, Name='David', Department='HR', Salary=5500)


## DataSet

In [58]:
from pyspark.sql import SparkSession, Row

# Create SparkSession
spark = SparkSession.builder.appName("MySparkApp").master("local[*]").getOrCreate()
sc = spark.sparkContext

# Create RDD of Row objects
rdd = sc.parallelize([Row(name="Alice", age=25), Row(name="Bob", age=30)])

# Convert RDD to DataFrame (PySpark equivalent of Dataset)
df = spark.createDataFrame(rdd)

# Filter: Age > 25
filtered = df.filter(df.age > 25)

# Select only 'name' column
names = filtered.select("name")

# Show results
names.show()


+----+
|name|
+----+
| Bob|
+----+



In [59]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SparkSQLBasics").getOrCreate()

data = [
    (1, "Alice", "Sales", 3000),
    (2, "Bob", "IT", 4000),
    (3, "Cathy", "HR", 3500),
    (4, "David", "Sales", 4500),
    (5, "Eva", "IT", 4200)
]
columns = ["EmpID", "Name", "Department", "Salary"]

df = spark.createDataFrame(data, columns)
df.show()

+-----+-----+----------+------+
|EmpID| Name|Department|Salary|
+-----+-----+----------+------+
|    1|Alice|     Sales|  3000|
|    2|  Bob|        IT|  4000|
|    3|Cathy|        HR|  3500|
|    4|David|     Sales|  4500|
|    5|  Eva|        IT|  4200|
+-----+-----+----------+------+



In [63]:
rdd=df.rdd
print("RDD Example:", rdd.map(lambda x: (x.Name, x.Salary)).collect())


RDD Example: [('Alice', 3000), ('Bob', 4000), ('Cathy', 3500), ('David', 4500), ('Eva', 4200)]


In [61]:
df.select("Name","Salary").show()

+-----+------+
| Name|Salary|
+-----+------+
|Alice|  3000|
|  Bob|  4000|
|Cathy|  3500|
|David|  4500|
|  Eva|  4200|
+-----+------+



In [62]:
df.createOrReplaceTempView("employees")

strSQL="select * from employees"
sql_result=spark.sql(strSQL)
sql_result.show()

+-----+-----+----------+------+
|EmpID| Name|Department|Salary|
+-----+-----+----------+------+
|    1|Alice|     Sales|  3000|
|    2|  Bob|        IT|  4000|
|    3|Cathy|        HR|  3500|
|    4|David|     Sales|  4500|
|    5|  Eva|        IT|  4200|
+-----+-----+----------+------+



## Data Format

In [64]:
data = [
    (1, "Alice", "Sales", 3000),
    (2, "Bob", "IT", 4000),
    (3, "Cathy", "HR", 3500)
]
columns = ["EmpID", "Name", "Department", "Salary"]
df = spark.createDataFrame(data, columns)
df.show()

+-----+-----+----------+------+
|EmpID| Name|Department|Salary|
+-----+-----+----------+------+
|    1|Alice|     Sales|  3000|
|    2|  Bob|        IT|  4000|
|    3|Cathy|        HR|  3500|
+-----+-----+----------+------+



In [65]:
pwd


'/content'

In [66]:
df.write.mode("overwrite").json("/content/json_data")

In [67]:
!ls /content/json_data/

part-00000-1c54e382-6920-4f6e-831b-4787a7ec6cd6-c000.json  _SUCCESS
part-00001-1c54e382-6920-4f6e-831b-4787a7ec6cd6-c000.json


In [68]:
!cat /content/json_data/part-00000-1c54e382-6920-4f6e-831b-4787a7ec6cd6-c000.json


{"EmpID":1,"Name":"Alice","Department":"Sales","Salary":3000}


In [69]:
!cat /content/json_data/part-00001-1c54e382-6920-4f6e-831b-4787a7ec6cd6-c000.json

{"EmpID":2,"Name":"Bob","Department":"IT","Salary":4000}
{"EmpID":3,"Name":"Cathy","Department":"HR","Salary":3500}


In [71]:
json_df= spark.read.json("/content/json_data/")
json_df.show()

+----------+-----+-----+------+
|Department|EmpID| Name|Salary|
+----------+-----+-----+------+
|        IT|    2|  Bob|  4000|
|        HR|    3|Cathy|  3500|
|     Sales|    1|Alice|  3000|
+----------+-----+-----+------+



In [73]:

StrPath="/content/json_data/"
df.write.mode("overwrite").option("header", "true").csv(StrPath)

In [74]:
csv_df=spark.read.option("header","true").csv(StrPath)
csv_df.show()

+-----+-----+----------+------+
|EmpID| Name|Department|Salary|
+-----+-----+----------+------+
|    2|  Bob|        IT|  4000|
|    3|Cathy|        HR|  3500|
|    1|Alice|     Sales|  3000|
+-----+-----+----------+------+



## Spark Streaming

In [1]:
import random
import csv

# Generate 30 records with random data
names = ["John", "Jane", "Mike", "Sara", "David", "Emily", "George", "Nina", "Tom", "Anna"]
departments = ["Sales", "IT", "HR", "Finance", "Marketing"]
salaries = [3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

# Create and open a CSV file for writing
with open('employee_data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the header
    writer.writerow(["ID", "Name", "Department", "Salary"])

    # Write the 30 records
    for i in range(1, 31):
        name = random.choice(names)
        department = random.choice(departments)
        salary = random.choice(salaries)
        writer.writerow([i, name, department, salary])

print("CSV file 'employee_data.csv' has been generated successfully.")

CSV file 'employee_data.csv' has been generated successfully.


In [2]:
!cat /content/employee_data.csv

ID,Name,Department,Salary
1,John,Sales,4000
2,Jane,Finance,6000
3,David,IT,3000
4,Emily,Marketing,3000
5,John,Sales,6000
6,George,Marketing,6000
7,Jane,Finance,10000
8,Emily,Sales,4000
9,Emily,HR,3000
10,Tom,Marketing,3000
11,Emily,Sales,8000
12,Nina,HR,3000
13,Sara,Sales,4000
14,Jane,Sales,8000
15,John,Finance,8000
16,Tom,IT,6000
17,John,IT,5000
18,David,Marketing,5000
19,John,Finance,3000
20,Jane,IT,10000
21,George,Sales,10000
22,Tom,Finance,6000
23,Emily,HR,3000
24,George,HR,8000
25,David,Marketing,4000
26,Anna,Marketing,6000
27,Emily,HR,3000
28,Jane,Marketing,9000
29,Tom,HR,4000
30,David,Marketing,10000


In [3]:
ls /content/ -a

./  ../  .config/  employee_data.csv  sample_data/


In [4]:
mkdir /content/empdata/

In [5]:
mv /content/employee_data.csv /content/empdata/

In [6]:
ls /content/empdata/

employee_data.csv


In [7]:
ls /content/empdata/employee_data.csv

/content/empdata/employee_data.csv


In [10]:
from pyspark.sql.types import StringType,StructType,IntegerType

schema = StructType() \
      .add("EmpID", IntegerType()) \
      .add("Name", StringType()) \
      .add("Department",StringType())\
      .add("Salary", IntegerType())
print(schema)

StructType([StructField('EmpID', IntegerType(), True), StructField('Name', StringType(), True), StructField('Department', StringType(), True), StructField('Salary', IntegerType(), True)])


In [12]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkSQLBasics").getOrCreate()
stream_df=spark.readStream \
   .option("sep",",") \
   .schema(schema) \
   .csv("/content/empdata/")

In [13]:
from pyspark.sql.functions import upper
transformed_df = stream_df.withColumn("NameUPPER",upper("Name"))

In [14]:
query = transformed_df.writeStream \
  .outputMode("append") \
  .format("console") \
  .start()


In [15]:
query.stop()

In [17]:
%%bash
cat<<EOF > /content/empdata.employee_data2.csv
1,John,Sales,3000
2,Jane,IT,4000
3,Mike,Sales,5000
4,Sara,Finance,6000
5,David,HR,7000
6,Emily,Marketing,6000
7,George,HR,4000
8,Nina,Sales,5000
9,Tom,IT,8000
10,Anna,Marketing,3000
EOF


In [18]:
%%bash
cat<<EOF > /content/empdata.employee_data3.csv
11,John,IT,7000
12,Jane,HR,4000
13,Mike,Finance,5000
14,Sara,Sales,6000
15,David,Marketing,7000
16,Emily,Sales,8000
17,George,Finance,3000
18,Nina,IT,6000
19,Tom,Sales,4000
20,Anna,HR,5000
EOF


In [19]:
%%bash
cat<<EOF > /content/empdata.employee_data4.csv
21,John,Finance,7000
22,Jane,Marketing,6000
23,Mike,HR,8000
24,Sara,Sales,3000
25,David,IT,6000
26,Emily,Finance,5000
27,George,Marketing,4000
28,Nina,HR,7000
29,Tom,Finance,5000
30,Anna,IT,8000
EOF

In [20]:
ls /content/empdata/

employee_data.csv


In [21]:
rm /content/empdata.employee_data2.csv

In [22]:
rm /content/empdata.employee_data3.csv

In [23]:
rm /content/empdata.employee_data4.csv

In [24]:
%%bash
cat<<EOF > /content/empdata/employee_data2.csv
1,John,Sales,3000
2,Jane,IT,4000
3,Mike,Sales,5000
4,Sara,Finance,6000
5,David,HR,7000
6,Emily,Marketing,6000
7,George,HR,4000
8,Nina,Sales,5000
9,Tom,IT,8000
10,Anna,Marketing,3000
EOF

In [25]:
%%bash
cat<<EOF > /content/empdata/employee_data3.csv
11,John,IT,7000
12,Jane,HR,4000
13,Mike,Finance,5000
14,Sara,Sales,6000
15,David,Marketing,7000
16,Emily,Sales,8000
17,George,Finance,3000
18,Nina,IT,6000
19,Tom,Sales,4000
20,Anna,HR,5000
EOF


In [26]:
%%bash
cat<<EOF > /content/empdata/employee_data4.csv
21,John,Finance,7000
22,Jane,Marketing,6000
23,Mike,HR,8000
24,Sara,Sales,3000
25,David,IT,6000
26,Emily,Finance,5000
27,George,Marketing,4000
28,Nina,HR,7000
29,Tom,Finance,5000
30,Anna,IT,8000
EOF

In [27]:
ls /content/empdata/

employee_data2.csv  employee_data3.csv  employee_data4.csv  employee_data.csv


In [28]:
query = transformed_df.writeStream \
  .outputMode("append") \
  .format("console") \
  .start()


In [29]:
query.awaitTermination(10)


False

In [42]:
query.stop()


In [37]:
import os, shutil

# Clean folder if exists
if os.path.exists("/content/empdata/employee_data2.csv"):
    shutil.rmtree("/content/empdata/employee_data2.csv")
os.makedirs("/content/empdata//employee_data2.csv", exist_ok=True)


In [38]:
query = transformed_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()


In [39]:
query.awaitTermination(15)  # wait 15 seconds for Spark to pick the file


False

In [43]:
query = transformed_df.writeStream \
    .outputMode("append") \
    .format("json") \
    .option("path", "/tmp/stream_json_output") \
    .option("checkpointLocation", "/tmp/stream_checkpoint") \
    .start()

In [47]:
import time
time.sleep(10) # Wait for the streaming query to write some data
df = spark.read.schema(transformed_df.schema).json("/tmp/stream_json_output")
df.show()

+-----+----+----------+------+---------+
|EmpID|Name|Department|Salary|NameUPPER|
+-----+----+----------+------+---------+
+-----+----+----------+------+---------+



In [48]:
ls /tep/

ls: cannot access '/tep/': No such file or directory


In [49]:
ls /tmp

blockmgr-6de53b31-6b6f-4ef5-b275-1054aa4f52bb/
colab_runtime.sock=
dap_multiplexer.6fee3d9b4636.root.log.INFO.20250730-091916.84
dap_multiplexer.INFO@
debugger_24r7hsa7kf=
hsperfdata_root/
initgoogle_syslog_dir.0/
language_service.6fee3d9b4636.root.log.ERROR.20250730-092716.378
language_service.6fee3d9b4636.root.log.INFO.20250730-092016.378
language_service.6fee3d9b4636.root.log.INFO.20250730-103456.18662
language_service.6fee3d9b4636.root.log.WARNING.20250730-092716.378
language_service.ERROR@
language_service.INFO@
language_service.WARNING@
liblz4-java-6707959208834418084.so
liblz4-java-6707959208834418084.so.lck
pyright-18668-3OeLvOrVL94o/
pyright-18668-Io5UCw3UXZwN/
pyright-386-3YZ4ciSdPApK/
python-languageserver-cancellation/
spark-305d9024-6a48-49c9-a0c7-8c6e72e52394/
spark-bdd5f8b1-044c-44d6-8b23-710e746c581b/
stream_checkpoint/
stream_json_output/
temporary-8d8581c4-3d9b-4741-998d-cd1471dd26fa/
temporary-95c3dd53-946d-4ff3-89d8-c3e51784255d/
temporary-e174db05-e63f-4d92-bb6c-22

In [50]:
!ls /tmp/stream_json_output

_spark_metadata


In [51]:
cat /tmp/stream_json_output/_spark_metadata/

cat: /tmp/stream_json_output/_spark_metadata/: Is a directory


In [52]:
stream_df = spark.readStream \
   .option("sep", ",") \
   .schema(schema) \
   .csv("/content/empdata/")


In [53]:
import time
time.sleep(5)


In [54]:
%%bash
cat <<EOF > /content/empdata/emp1.csv
1,John,Sales,3000
2,Jane,IT,4000
EOF


In [55]:
import time
time.sleep(5)


In [56]:
from pyspark.sql.types import StructType, StringType, IntegerType

schema_out = StructType() \
    .add("EmpID", IntegerType()) \
    .add("Name", StringType()) \
    .add("Department", StringType()) \
    .add("Salary", IntegerType()) \
    .add("NameUPPER", StringType())

df = spark.read.schema(schema_out).json("/tmp/stream_json_output")
df.show()


+-----+----+----------+------+---------+
|EmpID|Name|Department|Salary|NameUPPER|
+-----+----+----------+------+---------+
|    1|John|     Sales|  3000|     JOHN|
|    2|Jane|        IT|  4000|     JANE|
+-----+----+----------+------+---------+



In [57]:
pip install faker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 34.7 MB/s eta 0:00:00


In [59]:
from faker import Faker
import random
import time
import uuid

# Create Faker instance
fake = Faker()

# Output file
output_file = "/content/empdata/random_data.csv"

while True:
    emp_id = random.randint(1000, 9999)      # Random Employee ID
    name = fake.name()                       # Random Name
    department = fake.job()                  # Random Job Title as Department
    salary = random.randint(3000, 12000)     # Random Salary
    unique_id = uuid.uuid4()                 # Unique ID for no repetition

    record = f"{emp_id},{name},{department},{salary},{unique_id}\n"

    # Append to file
    with open(output_file, "a") as f:
        f.write(record)

    print("Generated:", record.strip())

    time.sleep(1)  # Wait 1 second before generating next line


Generated: 1694,Robert Cohen,Aid worker,3507,0a622e19-b9b2-4a33-b618-53504009d33a
Generated: 2800,Sean Morgan,Designer, television/film set,5710,fa395b0d-bce6-46bd-88eb-ddf69f35f542
Generated: 3511,Colleen Mullins,Meteorologist,7885,1cde45b0-d5b6-492b-8267-46d216ddb138
Generated: 6536,Marilyn Hensley,Immigration officer,11839,96cd3128-c4d6-4d8f-a140-f8d2cfd266fd
Generated: 5347,Bryan Miller,Chartered accountant,5064,4840673e-cf80-470e-8d24-abeb067168b4
Generated: 5799,Erica Terry,Therapist, speech and language,4851,f8cb301c-6b76-4682-98dd-f58777d009f0
Generated: 9016,Tammy Smith MD,Biomedical scientist,9702,1752df1d-8daa-4acd-92ca-ad0617db32f1
Generated: 2552,Jessica Mejia,Copy,10762,7b70a641-c634-4564-b730-b23cfcae1d9c
Generated: 3159,Robert Walker,Computer games developer,8403,fc75e1cf-accc-49d3-87bb-e961737f04cd
Generated: 4770,Keith Conley,Land/geomatics surveyor,3738,afdad4a0-841b-4c64-8dd0-9c8702dcb485
Generated: 6396,Mary Thomas,Teacher, music,11720,a9e085b9-23d2-46ed-b22f-a5016

KeyboardInterrupt: 

In [60]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [61]:
from faker import Faker
import random
import time
import uuid
import os

# Create Faker instance
fake = Faker()

# Set the file path inside Google Drive
output_dir = "/content/drive/My Drive/RandomData"
os.makedirs(output_dir, exist_ok=True)

output_file = os.path.join(output_dir, "stream_data.csv")

# Generate random data continuously
while True:
    emp_id = random.randint(1000, 9999)
    name = fake.name()
    department = fake.job()
    salary = random.randint(3000, 12000)
    unique_id = uuid.uuid4()

    record = f"{emp_id},{name},{department},{salary},{unique_id}\n"

    with open(output_file, "a") as f:
        f.write(record)

    print("Generated:", record.strip())

    time.sleep(1)  # 1 second delay


Generated: 1170,James Marquez,Therapist, speech and language,9904,f6e59a82-a575-47cc-b11b-50b0e5649667
Generated: 2114,Cheryl Johnson,Financial controller,4290,d737cc43-4f5f-463b-81c6-17223f54ca7d
Generated: 2465,Kara Jones,Conservator, furniture,9236,7b8ccfd7-4802-4aac-8c36-4a9d76a4bb9a
Generated: 1916,Michael Morgan,Manufacturing systems engineer,8534,30f9cead-7ff6-4188-a82c-b75661f209bf
Generated: 2441,John White,Public house manager,11324,2e9b800b-923a-4e9f-b12a-ee3b75c9f397
Generated: 5346,Willie Anderson,Scientist, forensic,3573,deb555a1-764c-4cf4-b642-d5a808b14d4c
Generated: 5528,Melissa Davenport,Archivist,11060,e839dd7e-bcbf-47bb-891f-74269d6615d1
Generated: 6362,Angela Hamilton,Stage manager,5243,ed931e34-d95a-471d-b44f-84c1b43ac1d7
Generated: 2983,Sharon Yang,Equality and diversity officer,7872,1e7d7caf-37ed-47a3-bdc9-da0d983199bd
Generated: 4092,Jennifer Lopez,Mechanical engineer,11564,7854b1ec-40c4-419a-8bc5-4d8479da58e0
Generated: 6163,Bailey Johnson,Surgeon,9473,b2d41bdf

KeyboardInterrupt: 

In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
from faker import Faker
import random
import uuid
import os
import time
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Setup
fake = Faker()
output_dir = "/content/drive/My Drive/RandomDataFiles"
os.makedirs(output_dir, exist_ok=True)

file_count = 1  # To number files

while True:
    # Generate 10 random records
    records = []
    for _ in range(10):
        emp_id = random.randint(1000, 9999)
        name = fake.name()
        department = fake.job()
        salary = random.randint(3000, 12000)
        unique_id = uuid.uuid4()
        record = f"{emp_id},{name},{department},{salary},{unique_id}"
        records.append(record)

    # Save to a new file
    file_path = os.path.join(output_dir, f"data_file_{file_count}.csv")
    with open(file_path, "w") as f:
        f.write("\n".join(records))

    print(f"✅ Created: {file_path} with 10 records")

    file_count += 1
    time.sleep(5)  # Wait 5 seconds before creating the next file


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Created: /content/drive/My Drive/RandomDataFiles/data_file_1.csv with 10 records
✅ Created: /content/drive/My Drive/RandomDataFiles/data_file_2.csv with 10 records
✅ Created: /content/drive/My Drive/RandomDataFiles/data_file_3.csv with 10 records
✅ Created: /content/drive/My Drive/RandomDataFiles/data_file_4.csv with 10 records
✅ Created: /content/drive/My Drive/RandomDataFiles/data_file_5.csv with 10 records
✅ Created: /content/drive/My Drive/RandomDataFiles/data_file_6.csv with 10 records
✅ Created: /content/drive/My Drive/RandomDataFiles/data_file_7.csv with 10 records
✅ Created: /content/drive/My Drive/RandomDataFiles/data_file_8.csv with 10 records
✅ Created: /content/drive/My Drive/RandomDataFiles/data_file_9.csv with 10 records
✅ Created: /content/drive/My Drive/RandomDataFiles/data_file_10.csv with 10 records
✅ Created: /content/drive/My Drive/Rando

KeyboardInterrupt: 